In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import openpyxl
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner
from keras_tuner import RandomSearch
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Flatten
from tensorflow.keras.optimizers import Adam
from openpyxl import load_workbook
import os

In [14]:
excel_file = pd.ExcelFile("pet_btech_data_location.xlsx")
for sheet_name in excel_file.sheet_names:
    if sheet_name == "location_1":
        sheet_data = excel_file.parse(sheet_name)
        print("Data in sheet 'location_1':")
        print(sheet_data)

Data in sheet 'location_1':
      idates  imonths  iyears  observeddata_77.5_8.5  meanmodeldata_77.5_8.5  \
0          1        1    2007                   4.08                    3.93   
1          2        1    2007                   4.63                    5.18   
2          3        1    2007                   4.84                    5.21   
3          4        1    2007                   4.64                    4.89   
4          5        1    2007                   4.74                    5.17   
...      ...      ...     ...                    ...                     ...   
4630      27       12    2019                   3.87                    4.07   
4631      28       12    2019                   4.49                    4.73   
4632      29       12    2019                   3.89                    4.08   
4633      30       12    2019                   4.22                    4.48   
4634      31       12    2019                   3.95                    4.20   

      stdmo

In [15]:

def create_lstm_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('first_hidden_layer_units', min_value=64, max_value=256, step=32),
                   activation='relu',
                   input_shape=(1, 3),
                   return_sequences=True))
    
    model.add(LSTM(units=hp.Int('second_hidden_layer_units', min_value=32, max_value=128, step=16),
                   activation='relu',
                   return_sequences=True))
    
    model.add(LSTM(units=hp.Int('third_hidden_layer_units', min_value=16, max_value=64, step=8),
                   activation='relu',
                   return_sequences=True))
    
    model.add(Flatten())  # Flatten before Dense layer
    
    model.add(Dense(units=hp.Int('dense_layer_units', min_value=8, max_value=64, step=8), activation='relu'))
    
    model.add(Dense(1))
    
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5]))
    
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return model


In [19]:
def train_and_evaluate_model(X_train, Y_train, X_test, Y_test, sheet_name, split_num):

   #Creating a separate directory for each split
    split_directory = os.path.join('RandomSearch', sheet_name, f'split_{split_num}')
    os.makedirs(split_directory, exist_ok=True)

    tuner = RandomSearch(
        create_lstm_model,
        objective= "mean_squared_error",
        max_trials=50,
        executions_per_trial=1,
        directory=split_directory,  # Directory to store results
        project_name = 'BTP',
        overwrite=False
    )
    results=[]

    kf = KFold(n_splits=5, shuffle=False)

    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[val_index]
      
        tuner.search(X_train_fold, Y_train_fold, validation_data=(X_val_fold, Y_val_fold), epochs=100, batch_size=32)
        
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)
    print(best_hps.get_config())
    
    history = best_model.fit(X_train, Y_train, epochs=101, validation_data=(X_val_fold, Y_val_fold), batch_size=32, verbose=1)

    Y_pred = best_model.predict(X_test)
    Y_pred = Y_pred.flatten()

        # Calculating RMSE, MSE, MAE, and bias for "model_trained_observed_value"
    rmse = np.sqrt(mean_squared_error(Y_test,Y_pred))
    mse = mean_squared_error(Y_test,Y_pred)
    mae = mean_absolute_error(Y_test,Y_pred)
    bias = np.mean(Y_test- Y_pred)

    results.append({
            'RMSE': rmse,
            'MSE': mse,
            'MAE': mae,
            'Bias': bias
        })

    return results, Y_pred

In [22]:
import csv
import re

# Initializing a list to store all metrics
output_directory = "C:/Users/Hello/OneDrive/Desktop/BTP"
all_metrics = []
for sheet_name in excel_file.sheet_names[:10]:
    df = excel_file.parse(sheet_name)

    meanmodeldata_column = [col for col in df.columns if re.match(r'meanmodeldata_\d+\.\d+_\d+\.\d+', col)]
    observeddata_column = [col for col in df.columns if re.match(r'observeddata_\d+\.\d+_\d+\.\d+', col)]

    if meanmodeldata_column and observeddata_column:
        meanmodeldata_column = meanmodeldata_column[0]  
        observeddata_column = observeddata_column[0]  
    else:
        print(f"Sheet '{sheet_name}' does not contain the expected columns with the pattern.")
        continue

    meanmodeldata = df[meanmodeldata_column].values
    observeddata = df[observeddata_column].values
    
    idates = df['idates'].values
    imonths = df['imonths'].values
    iyears = df['iyears'].values

    df['Date'] = pd.to_datetime(df[['idates', 'imonths', 'iyears']].astype(str).agg('/'.join, axis=1), format='%d/%m/%Y')
    df.drop(columns=['idates', 'imonths', 'iyears'], inplace=True)
    Date = df['Date'].values

    df['julean_day'] = pd.to_datetime(df['Date']).dt.dayofyear

    feature_column = [col for col in df.columns if "meanmodeldata" in col or "stdmodeldata" in col or "julean_day" in col]
    target_column = [col for col in df.columns if "observeddata" in col][0]  


    # Split the data into training and testing sets (4-way split)

    num_splits = 4
    split_size = len(df) // num_splits
   
    sheet_average_metrics = []
    all_results= []
        
    for split_num in range(num_splits):
        start_index = split_num * split_size
        end_index = (split_num + 1) * split_size

        X = df[feature_column].values
        Y = df[target_column].values

        # Use indices to perform sequential split
        X_train, X_test = np.concatenate([X[:start_index], X[end_index:]]), X[start_index:end_index]
        Y_train, Y_test = np.concatenate([Y[:start_index], Y[end_index:]]), Y[start_index:end_index]

        meanmodeldata_test = meanmodeldata[start_index:end_index]
        observeddata_test = observeddata[start_index:end_index]
        
        print(meanmodeldata_test)
        print(observeddata_test)

        Date_test = df['Date'].values[start_index:end_index]
        
        # Reshaping the input for LSTM
        X_train = X_train.reshape(X_train.shape[0], 1, 3)
        X_test = X_test.reshape(X_test.shape[0], 1, 3)

        print('length of X_test =', len(X_test))
        print('length of X_train =', len(X_train))
        print('length of Y_test =', len(Y_test))
        print('length of Y_train =', len(Y_train))

        print('sheet name', sheet_name, ':' , 'num_split', split_num, ': ',"X_train" , X_train  )
        print('sheet name', sheet_name, ':' , 'num_split', split_num, ': ', "X_test",  X_test )

        results, Y_pred= train_and_evaluate_model(X_train, Y_train, X_test, Y_test, sheet_name, split_num)
        sheet_average_metrics.append(results)

        
        for i in range(len(Y_test)):
                    all_results.append({
                        'Sheet Name': sheet_name,
                        'Split Number': split_num,
                        'Date': Date_test[i],
                        'Model_trained_observed_value': Y_pred[i],
                        'observeddata_test': Y_test[i],
                        'meanmodeldata_test' : meanmodeldata_test[i]
                    })

        #  metrics for the current split
        print(f"Results for {sheet_name} - Split {split_num+1}")
        for res in results:
            print(f"RMSE: {res['RMSE']:.4f}")
            print(f"MSE: {res['MSE']:.4f}")
            print(f"MAE: {res['MAE']:.4f}")
            print(f"Bias: {res['Bias']:.4f}")
            print()

    # To Calculate the average metrics for this sheet
    # Initializing lists to store each metric separately
    rmse_values = []
    mse_values = []
    mae_values = []
    bias_values = []

    for result in sheet_average_metrics:
        for res in result:
            rmse_values.append(res['RMSE'])
            mse_values.append(res['MSE'])
            mae_values.append(res['MAE'])
            bias_values.append(res['Bias'])

    # mean for each metric
    average_rmse = np.mean(rmse_values)
    average_mse = np.mean(mse_values)
    average_mae = np.mean(mae_values)
    average_bias = np.mean(bias_values)

    # Average metrics for the current sheet
    print(f"Average Metrics for {sheet_name}:")
    print(f"Average RMSE: {average_rmse:.4f}")
    print(f"Average MSE: {average_mse:.4f}")
    print(f"Average MAE: {average_mae:.4f}")
    print(f"Average Bias: {average_bias:.4f}")
    print()

    #Metrics for Raw Data
    
    mse_raw = np.mean((observeddata-meanmodeldata) ** 2)
    rmse_raw = np.sqrt(mse_raw)
    mae_raw = np.mean(np.abs(observeddata-meanmodeldata))
    bias_raw = np.mean(observeddata-meanmodeldata)

    print(f"Metrics of Raw Data {sheet_name}:")
    print(f"RMSE: {rmse_raw:.4f}")
    print(f"MSE: {mse_raw:.4f}")
    print(f"MAE: {mae_raw:.4f}")
    print(f"Bias: {bias_raw:.4f}")
    print()

    # Append the average metrics to the all_metrics list
    all_metrics.append([sheet_name, average_rmse, average_mse, average_mae, average_bias,rmse_raw, mse_raw, mae_raw, bias_raw ])

    # Write all metrics to a CSV file
    output_file_path = os.path.join(output_directory, 'error.csv')
    with open(output_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write the header row
        writer.writerow(['Sheet Name', 'Average RMSE', 'Average MSE', 'Average MAE', 'Average Bias','rmse_raw', 'mse_raw', 'mae_raw', 'bias_raw'])
        # Write the data rows
        writer.writerows(all_metrics)

    # Save all results to a single CSV file
    result_file = os.path.join(output_directory, f'{sheet_name}.csv')
    result_df = pd.DataFrame(all_results)
    result_df.to_csv(result_file, index=False)


[3.93 5.18 5.21 ... 5.96 6.88 7.07]
[4.08 4.63 4.84 ... 5.38 5.98 5.96]
length of X_test = 1158
length of X_train = 3477
length of Y_test = 1158
length of Y_train = 3477
sheet name location_1 : num_split 0 :  X_train [[[6.88e+00 8.00e-02 7.80e+01]]

 [[6.72e+00 1.00e-01 7.90e+01]]

 [[6.07e+00 9.00e-02 8.00e+01]]

 ...

 [[4.08e+00 1.60e-01 3.63e+02]]

 [[4.48e+00 1.30e-01 3.64e+02]]

 [[4.20e+00 1.50e-01 3.65e+02]]]
sheet name location_1 : num_split 0 :  X_test [[[ 3.93  0.09  1.  ]]

 [[ 5.18  0.08  2.  ]]

 [[ 5.21  0.1   3.  ]]

 ...

 [[ 5.96  0.09 75.  ]]

 [[ 6.88  0.08 76.  ]]

 [[ 7.07  0.11 77.  ]]]
Reloading Tuner from RandomSearch\location_1\split_0\BTP\tuner0.json
{'space': [{'class_name': 'Int', 'config': {'name': 'first_hidden_layer_units', 'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 32, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'second_hidden_layer_units', 'default': None, 'conditions': [], 'min_value': 32, 'max_v

OSError: [Errno 28] No space left on device